In [20]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict,Literal,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from dotenv import load_dotenv

In [21]:
load_dotenv()

True

In [22]:
class ChatbotState(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]

In [23]:
llm = ChatOpenAI(model="gpt-4o-mini")

def chat_node(state:ChatbotState):
    
    response = llm.invoke(state["messages"])
    return {"messages":[response]}


In [24]:
# add checkpointer to utilize local memory for chat history
checkpointer = MemorySaver()
# create graph with nodes and edges
graph = StateGraph(ChatbotState)

graph.add_node("chat_node",chat_node)

graph.add_edge(START,"chat_node")
graph.add_edge("chat_node",END)
# compile graph to create chatbot object
chatbot = graph.compile(checkpointer=checkpointer)

In [25]:
# create a thread id for chat history
thread_id = "1"
# start chatting with AI chatbot
while True:
    user_query = input("type here:")
    print(f"User: {user_query}")
    
    # loop exit criteria
    if user_query.strip().lower() in ['exit','bye','end','quit']:
        break 

    #create config for memory allocation to chat history
    config = {'configurable':{'thread_id':thread_id}}
    response = chatbot.invoke({"messages": [HumanMessage(content=user_query)]},config=config)
    print(f"AI: {response['messages'][-1].content}")

    


User: Hi,my name is Zufeshan
AI: Hello Zufeshan! How can I assist you today?
User: what is my name?
AI: Your name is Zufeshan. How can I assist you further?
User: Can you add 100 to 200
AI: Sure! Adding 100 to 200 gives you 300.
User: now multiply the result by 2
AI: Multiplying 300 by 2 gives you 600.
User: awesome !, you remember the conversation
AI: I don't have the ability to remember past interactions once the conversation ends. However, I can reference what we've talked about during our current session. If you have any other questions or need assistance, feel free to ask!
User: thank you
AI: You're welcome, Zufeshan! If you have any more questions or need help with anything else, just let me know.
User: end
